In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

pd.options.display.float_format = '{:.4f}'.format
import statsmodels.api as sm

**For this model I analayzed the heatmap/ correlations ofnumeric columns with sale price and the catterplots/ bargraphs of the categorical columns and sale price. I will start witha model with just these variables and then use stepwise regression to add and drop variables**

In [5]:
clean_data = pd.read_csv('../datasets/clean_df.csv')

In [6]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       2051 non-null   int64  
 1   Id               2051 non-null   int64  
 2   PID              2051 non-null   int64  
 3   MS SubClass      2051 non-null   int64  
 4   MS Zoning        2051 non-null   object 
 5   Lot Frontage     1721 non-null   float64
 6   Lot Area         2051 non-null   int64  
 7   Street           2051 non-null   object 
 8   Alley            140 non-null    object 
 9   Lot Shape        2051 non-null   object 
 10  Land Contour     2051 non-null   object 
 11  Utilities        2051 non-null   object 
 12  Lot Config       2051 non-null   object 
 13  Land Slope       2051 non-null   object 
 14  Neighborhood     2051 non-null   object 
 15  Condition 1      2051 non-null   object 
 16  Condition 2      2051 non-null   object 
 17  Bldg Type     

In [7]:
Columns_to_include = ['SalePrice', 
                      'Overall Qual', 
                      'Gr Liv Area', 
                      'Garage Area', 
                      'Garage Cars',  
                      'Full Bath', 
                      'Fireplaces', 
                      'MS SubClass', 
                      'Neighborhood', 
                      'Condition 1', 
                      'Exter Qual', 
                      'Bsmt Exposure', 
                      'Kitchen Qual', 
                      'Garage Qual',
                      'Exterior 1st',
                      'Exter Cond',
                      'Bsmt Qual',
                      'Bsmt Cond',
                      'BsmtFin Type 1',
                      'Functional',
                      'Fireplace Qu',
                      'Paved Drive',
                      'Sale Type',
                      'Garage Cond']

In [8]:
clean_columns = clean_data[Columns_to_include]

In [9]:
# Create a list of string-type column names
string_columns = clean_data.select_dtypes(include=['object']).columns

column_to_exclude = 'Mas Vnr Type'

# Use a for loop to input NA in each column in dataframe that is in list except Mas Vnr Type
for column in string_columns:
    if column != column_to_exclude:
        clean_data[column].fillna('NA', inplace=True)

In [10]:
X1 = clean_data[Columns_to_include].drop(columns='SalePrice')
y = clean_data['SalePrice']

In [11]:
X1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Overall Qual    2051 non-null   int64  
 1   Gr Liv Area     2051 non-null   int64  
 2   Garage Area     2050 non-null   float64
 3   Garage Cars     2050 non-null   float64
 4   Full Bath       2051 non-null   int64  
 5   Fireplaces      2051 non-null   int64  
 6   MS SubClass     2051 non-null   int64  
 7   Neighborhood    2051 non-null   object 
 8   Condition 1     2051 non-null   object 
 9   Exter Qual      2051 non-null   object 
 10  Bsmt Exposure   2051 non-null   object 
 11  Kitchen Qual    2051 non-null   object 
 12  Garage Qual     2051 non-null   object 
 13  Exterior 1st    2051 non-null   object 
 14  Exter Cond      2051 non-null   object 
 15  Bsmt Qual       2051 non-null   object 
 16  Bsmt Cond       2051 non-null   object 
 17  BsmtFin Type 1  2051 non-null   o

In [12]:
X1['Garage Qual'].value_counts()

TA    1832
NA     114
Fa      82
Gd      18
Ex       3
Po       2
Name: Garage Qual, dtype: int64

In [13]:
y.shape

(2051,)

In [14]:
X1_train, X1_val, y_train, y_val = train_test_split(X1, y, test_size= 0.2, random_state=24)

In [15]:
X1_str = list(X1.select_dtypes(include=['object']).columns)

In [16]:
#I can either add imputer to the column transformer
#or if I don't add polynomical features back in, I can just remove ct & run each separate
#
ct = ColumnTransformer([('oh', OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first'), X1_str)
], remainder='passthrough')  

# Fit and transform the training data using the ColumnTransformer
X1_train_transformed = ct.fit_transform(X1_train)

# Transform the test data using the fitted ColumnTransformer
X1_val_transformed = ct.transform(X1_val)


#Double-check this... why would I need an imputer? I shouldn't have any numeric columns with missings
imputer = SimpleImputer(strategy='median')
X1_train_transformed = imputer.fit_transform(X1_train_transformed)
X1_val_transofrmed = imputer.transform(X1_val_transformed)


In [17]:
lr=LinearRegression()

In [18]:
lr.fit(X1_train_transformed, y_train)

LinearRegression()

In [19]:
r2_train = lr.score(X1_train_transformed, y_train)
r2_val = lr.score(X1_val_transformed, y_val)
print(r2_train)
print(r2_val)

0.8873055847891408
0.904916025625044


In [20]:
y_preds_train = lr.predict(X1_train_transformed)
metrics.mean_squared_error(y_train, y_preds_train)

716612922.1967893

In [21]:
y_preds_val = lr.predict(X1_val_transformed)
metrics.mean_squared_error(y_val, y_preds_val)

566641607.3660567

In [22]:
lr.coef_

array([-6.28114465e+03,  8.91854338e+03, -6.65428944e+03,  1.40004963e+04,
        3.88139284e+03,  1.84965394e+04, -1.30570005e+04,  2.88953046e+03,
        5.27676515e+03,  1.30335344e+05, -9.59020145e+03, -5.34982652e+02,
       -8.30729207e+03,  2.05362372e+02, -4.58151442e+03,  1.40112048e+03,
       -5.71973546e+01,  5.33054169e+04,  3.99889271e+04, -1.29614771e+04,
       -3.71478174e+03, -6.45414670e+02,  3.47974122e+03,  1.87635716e+04,
        5.87878015e+04,  9.52369006e+03,  1.27755835e+04,  1.98111600e+03,
        1.19089074e+04,  2.56074000e+04,  2.44672413e+04, -5.46742615e+03,
        1.26370251e+04, -8.71656392e+03,  4.91750548e+03, -3.28704285e+04,
       -2.37015785e+04, -2.74201105e+04,  1.84701975e+04, -7.74417395e+03,
        8.13657739e+03, -9.85755318e+03, -3.64312789e+04, -2.63283037e+04,
       -3.21637372e+04, -1.64159633e+05, -1.18561078e+05,  9.43487675e+03,
       -2.04593213e+05, -1.62485415e+05,  3.34170942e+04, -9.37219336e+03,
        2.46991102e+04,  

*Credit to Susan for add_constant to simplify creating an intercept process*

In [23]:
ols = sm.OLS(y_train, sm.add_constant(X1_train_transformed)).fit()
summary_table = ols.summary()
summary_table

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     104.3
Date:                Thu, 05 Oct 2023   Prob (F-statistic):               0.00
Time:                        17:33:54   Log-Likelihood:                -19047.
No. Observations:                1640   AIC:                         3.833e+04
Df Residuals:                    1524   BIC:                         3.895e+04
Df Model:                         115                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7.725e+04   4.01e+04      1.927      0.054   -1383.150    1.56e+05
x1         -6281.1446   1.47e+04     -0.426      0.670   -3.52e+04    2.26e+04
x2          8918.5434   1.11e+04      0.801      0.423   -1.29e+04    3.08e+04
x3         -6654.2894   8941.312     -0.744      0.457   -2.42e+04    1.09e+04
x4            1.4e+04   1.09e+04      1.286      0.199   -7359.601    3.54e+04
x5          3881.3928   7896.147      0.492      0.623   -1.16e+04    1.94e+04
x6           1.85e+04   8729.899      2.119      0.034    1372.652    3.56e+04
x7         -1.306e+04   8484.216     -1.539      0.124   -2.97e+04    3584.974
x8          2889.5305   7977.714      0.362      0.717   -1.28e+04    1.85e+04
x9          5276.7652   2.14e+04      0.246      0.805   -3.67e+04    4.73e+04
x10         1.303e+05   2.14e+04      6.087      0.000    8.83e+04    1.72e+05
x11        -9590.2015   9214.952     -1.041      0.298   -2.77e+04    8485.128
x12         -534.9827   2.89e+04     -0.018      0.985   -5.73e+04    5.62e+04
x13        -8307.2921    1.1e+04     -0.753      0.452   -2.99e+04    1.33e+04
x14          205.3624   8550.430      0.024      0.981   -1.66e+04     1.7e+04
x15        -4581.5144   8274.773     -0.554      0.580   -2.08e+04    1.16e+04
x16         1401.1205   1.11e+04      0.126      0.899   -2.03e+04    2.31e+04
x17          -57.1974   8672.628     -0.007      0.995   -1.71e+04     1.7e+04
x18         5.331e+04   9163.829      5.817      0.000    3.53e+04    7.13e+04
x19         3.999e+04   8155.959      4.903      0.000     2.4e+04     5.6e+04
x20        -1.296e+04   8561.821     -1.514      0.130   -2.98e+04    3832.721
x21        -3714.7817   1.04e+04     -0.357      0.721   -2.41e+04    1.67e+04
x22         -645.4147   8713.555     -0.074      0.941   -1.77e+04    1.64e+04
x23         3479.7412   8361.629      0.416      0.677   -1.29e+04    1.99e+04
x24         1.876e+04   8000.370      2.345      0.019    3070.670    3.45e+04
x25         5.879e+04   9058.077      6.490      0.000     4.1e+04    7.66e+04
x26         9523.6901   8843.041      1.077      0.282   -7822.128    2.69e+04
x27         1.278e+04   1.12e+04      1.142      0.253   -9161.639    3.47e+04
x28         1981.1160   5147.333      0.385      0.700   -8115.489    1.21e+04
x29         1.191e+04   4133.708      2.881      0.004    3800.548       2e+04
x30         2.561e+04   9627.984      2.660      0.008    6721.899    4.45e+04
x31         2.447e+04   7104.017      3.444      0.001    1.05e+04    3.84e+04
x32        -5467.4262   8229.337     -0.664      0.507   -2.16e+04    1.07e+04
x33         1.264e+04   6801.668      1.858      0.063    -704.595     2.6e+04
x34        -8716.5639   2.06e+04     -0.422      0.673   -4.92e+04    3.18e+04
x35         4917.5055   1.49e+04      0.330      0.742   -2.43e+04    3.42e+04
x3

In [24]:
ct.get_feature_names_out()

array(['oh__Neighborhood_Blueste', 'oh__Neighborhood_BrDale',
       'oh__Neighborhood_BrkSide', 'oh__Neighborhood_ClearCr',
       'oh__Neighborhood_CollgCr', 'oh__Neighborhood_Crawfor',
       'oh__Neighborhood_Edwards', 'oh__Neighborhood_Gilbert',
       'oh__Neighborhood_Greens', 'oh__Neighborhood_GrnHill',
       'oh__Neighborhood_IDOTRR', 'oh__Neighborhood_Landmrk',
       'oh__Neighborhood_MeadowV', 'oh__Neighborhood_Mitchel',
       'oh__Neighborhood_NAmes', 'oh__Neighborhood_NPkVill',
       'oh__Neighborhood_NWAmes', 'oh__Neighborhood_NoRidge',
       'oh__Neighborhood_NridgHt', 'oh__Neighborhood_OldTown',
       'oh__Neighborhood_SWISU', 'oh__Neighborhood_Sawyer',
       'oh__Neighborhood_SawyerW', 'oh__Neighborhood_Somerst',
       'oh__Neighborhood_StoneBr', 'oh__Neighborhood_Timber',
       'oh__Neighborhood_Veenker', 'oh__Condition 1_Feedr',
       'oh__Condition 1_Norm', 'oh__Condition 1_PosA',
       'oh__Condition 1_PosN', 'oh__Condition 1_RRAe',
       'oh__Condition

In [25]:
test = pd.read_csv('datasets/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/test.csv'

In [ ]:
test.info()

In [ ]:
#For the columns I am using, the solution for missing values was to fill na with 
test.fillna('NA', inplace=True)

In [ ]:
test_columns = ['Overall Qual', 
                'Gr Liv Area', 
                'Garage Area', 
                'Garage Cars', 
                'Full Bath', 
                'Fireplaces', 
                'MS SubClass', 
                'Neighborhood', 
                'Condition 1', 
                'Exter Qual', 
                'Bsmt Exposure', 
                'Kitchen Qual', 
                'Garage Qual',
                'Exterior 1st',
                'Exter Cond',
                'Bsmt Qual',
                'Bsmt Cond',
                'BsmtFin Type 1',
                'Functional',
                'Fireplace Qu',
                'Paved Drive',
                'Sale Type',
                'Garage Cond']
X_test = test[test_columns]

In [ ]:
X_test.info()

In [ ]:
#For Garage Qual & Bsmt Exposure, I can input NA for missing values as I did for training data

In [ ]:
X_test_transformed = ct.transform(X_test)

In [ ]:
saleprice = lr.predict(X_test_transformed)

In [ ]:
saleprice.shape

In [ ]:
data = {'Id':test['Id'], 'SalePrice': saleprice}
test_sub = pd.DataFrame(data)

In [ ]:
test_sub.shape

In [ ]:
test_sub.to_csv('datasets/submission4.csv', index=False)